In [53]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from datetime import datetime

평가는 RMSLE 로  
0에 가까울수록 좋다.

랜덤포레스트를 이용해서 풍속을 채워넣기  
자전거가 몇대가 필요할 것이다도 랜덤포레스트  
RandomForest 를 회기로도 사용 가능하다.

In [98]:
train=pd.read_csv("bike-sharing-demand/train.csv", parse_dates=['datetime'])
test=pd.read_csv("bike-sharing-demand/test.csv",parse_dates=['datetime'])

In [99]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   

### 전처리는 타이타닉에서 배웠음으로 오늘은 feature engineering 위주
### feature engineering

In [100]:
train['year']=train.datetime.dt.year
train['month']=train.datetime.dt.month
train['day']=train.datetime.dt.day
train['hour']=train.datetime.dt.hour
train['minute']=train.datetime.dt.minute
train['second']=train.datetime.dt.second
train['dayofweek']=train.datetime.dt.dayofweek

In [101]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour,minute,second,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0,0,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1,0,0,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2,0,0,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3,0,0,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4,0,0,5


In [102]:
test['year']=test.datetime.dt.year
test['month']=test.datetime.dt.month
test['day']=test.datetime.dt.day
test['hour']=test.datetime.dt.hour
test['minute']=test.datetime.dt.minute
test['second']=test.datetime.dt.second
test['dayofweek']=test.datetime.dt.dayofweek

In [103]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour,minute,second,dayofweek
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,0,0,0,3
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,1,0,0,3
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,2,0,0,3
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,0,0,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,4,0,0,3


In [104]:
train.shape , test.shape #train data가 열이 3개가 더 있다. casual, registered, count

((10886, 19), (6493, 16))

In [105]:
#평균으로 대체하는것은 좋은 방법이 아니다

### 좋은방법

In [106]:
#머신러닝(랜덤포레스트)를 이용한 풍속 0 값을 대체
#풍속 0, not 0 으로 분류
trainWind0=train.loc[train.windspeed==0]
trainWindNot0=train.loc[train.windspeed!=0]
print(trainWind0.shape) #(1313, 19)
# print(trainWindNot0)

(1313, 19)


In [107]:
train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'year', 'month', 'day', 'hour', 'minute', 'second', 'dayofweek'],
      dtype='object')

In [108]:
def predictWindSpeed(data):
    #data의 windspeed열 값이 0을 랜덤포레스트 기반 예측값으로 대체
    dataWind0=data.loc[data['windspeed']==0]
    dataWindNot0=data.loc[data['windspeed']!=0]
    #입력데이터 -> 랜덤포레스트 모델 -> 출력데이터(windspeed)
    #풍속을 예측하는데 사용될 변수(입력변수)를 선택(찾아야함) <가장 좋은것은 상관계수를 찾아보는것임>
    wCol=['season', 'weather','temp','atemp', 'humidity', 'year', 'month'] #이정도의 데이터는 부족한것임. 외부데이터를 사용하면 좋음
#     회귀모델
#     - 풍속예측함수: w1*season+w2+weather*+...+w7*month+b
#     - 중요: w와 b

#   출력 데이터 타입이 str이어야 했음.
    dataWindNot0['windspeed']=dataWindNot0['windspeed'].astype('str') #error handling
    
    #랜덤포레스트 분류기
    rfModelWind=RandomForestClassifier()
    
    
#     모델링(학습용 사용: windNot0사용을 해야 찾을수 있지)
#     wind not 0 데이터로 학습을 시켜서 모델을 만든 후, wind 0 데이터를 모델에 입력하면 예상되는 풍속이 출력된다.
#     rfModelWind.fit(input, output)
    rfModelWind.fit(dataWindNot0[wCol], dataWindNot0['windspeed'])
    
    #모델 (rfModelWind)을 이용하여 풍속이 0인 데이터에 대한 풍속을 예측(predict)하자.
    wind0Values=rfModelWind.predict(dataWind0[wCol])
    
    predictWind0=dataWind0
    predictWindNot0=dataWindNot0
    
    #풍속 0을 예측된 값으로 변경
    predictWind0['windspeed']=wind0Values
    
    #풍속이 0이 아닌 데이터프레임에 예측된 값이 저장된 데이터프레임을 합침
    data=predictWindNot0.append(predictWind0)
    
    #타입변경!
    data['windspeed']=data['windspeed'].astype('float')
    
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)

    
    return data
    

In [109]:
train=predictWindSpeed(train)
# test=predictWindSpeed(test)

<ipython-input-108-a9631868885d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataWindNot0['windspeed']=dataWindNot0['windspeed'].astype('str') #error handling
<ipython-input-108-a9631868885d>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictWind0['windspeed']=wind0Values


In [110]:
train.windspeed

0         6.0032
1        16.9979
2        19.0012
3        19.0012
4        19.9995
          ...   
10881     7.0015
10882     8.9981
10883     8.9981
10884    12.9980
10885     6.0032
Name: windspeed, Length: 10886, dtype: float64

In [111]:
train.windspeed.describe()

count    10886.000000
mean        14.022115
std          7.049856
min          6.003200
25%          8.998100
50%         12.998000
75%         19.001200
max         56.996900
Name: windspeed, dtype: float64

In [112]:
#연속형 변수 -> 범주형 변수 변환작업
category_fn=['season', 'holiday', 'workingday', 'weather', 'year', 'month', 'hour', 'dayofweek'] 
#성격상 범주형 변수인데 int로 되어있기 때문에 변환해야함


In [113]:
for v in category_fn:
    train[v]=train[v].astype('category')
    test[v]=test[v].astype('category')

In [114]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  category      
 2   holiday     10886 non-null  category      
 3   workingday  10886 non-null  category      
 4   weather     10886 non-null  category      
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  category      
 13  month       10886 non-null  category      
 14  day         10886 non-null  int64         
 15  hour        10886 non-null  category      
 16  minute      10886 non-

In [115]:
#RMSLE
from sklearn.metrics import make_scorer

In [116]:
def rmsle(pv, av): #예측값, 실제값
    #넘파이 배열로 변환
    pv=np.array(pv)
    av=np.array(av)
    #예측값과 실제값에 1을 더하고 로그를 씌운다
    log_predict=np.log(pv+1)
    log_actual=np.log(av+1)
    
    res=log_predict-log_actual
    res=np.square(res)
    res.mean() #합하는작업 따로 필요없음 평균을 구할때 자동임
    
    mean_res=res.mean()
    score=np.sqrt(mean_res)
    return score

In [117]:
rmsle_scorer=make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [118]:
#kfold 교차검증
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [119]:
kfold=KFold(n_splits=10, shuffle=True, random_state=42)

In [120]:
from sklearn.ensemble import RandomForestRegressor

In [121]:
model=RandomForestRegressor(n_estimators=100,
                     n_jobs=-1,
                     random_state=42)

In [133]:
#데이터 분리:
# fn=['season', 'holiday', 'workingday', 'weather', 'year', 'month', 
#     'hour', 'dayofweek', 'temp', 'atemp', 'humidity', 'windspeed'] 
#데이터 분리:
fn=['season', 'holiday', 'workingday', 'weather', 'year', 
    'hour', 'dayofweek', 'temp', 'atemp', 'humidity', 'windspeed'] 

In [134]:
xTrain=train[fn]
xTrain.shape #(10886, 12)

(10886, 11)

In [135]:
xTest=test[fn]
xTest.shape #(6493, 12)

(6493, 11)

In [136]:
yTrain=train['count']
yTrain.shape #(10886,)

(10886,)

In [137]:
model.fit(xTrain, yTrain)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [138]:
pred=model.predict(xTest)
pred

array([ 12.37,   5.02,   3.88, ...,  98.8 , 103.66,  46.52])

# KFold 교차검증

In [139]:
score=cross_val_score(model, xTrain, yTrain, cv=kfold, scoring=rmsle_scorer)
score #rmsle이므로 0에 가까울수록 잘나오는것임

array([0.29155285, 0.32154332, 0.34767709, 0.33027109, 0.33004261,
       0.32764872, 0.35636476, 0.33546937, 0.31753036, 0.35326078])

In [140]:
score.mean() #교차 검증을 통해서 위의 전처리를 예측한 것이 얼마나 일반화가 잘되어 있는지 확인하는것임.

0.33113609509584624

In [141]:
bikesubmit=pd.read_csv("bike-sharing-demand/sampleSubmission.csv")
bikesubmit

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0
...,...,...
6488,2012-12-31 19:00:00,0
6489,2012-12-31 20:00:00,0
6490,2012-12-31 21:00:00,0
6491,2012-12-31 22:00:00,0


In [142]:
bikesubmit['count']=pred
bikesubmit

,datetime,count
0,2011-01-20 00:00:00,12.37
1,2011-01-20 01:00:00,5.02
2,2011-01-20 02:00:00,3.88
3,2011-01-20 03:00:00,3.47
4,2011-01-20 04:00:00,2.86
...,...,...
6488,2012-12-31 19:00:00,211.98
6489,2012-12-31 20:00:00,143.77
6490,2012-12-31 21:00:00,98.80
6491,2012-12-31 22:00:00,103.66


In [143]:
bikesubmit.to_csv("bikesubmit.csv", index=False)